# 02 — RFM Segmentation (SQL‑first)

In [4]:
import os, duckdb, pandas as pd
from pathlib import Path

# 项目根目录（假设当前 notebook 在 notebooks/ 下运行）
BASE = Path("..").resolve()

# ✅ 确保 processed 目录存在
processed_dir = BASE / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

# ✅ 用绝对路径连接 DuckDB 文件数据库
db_path = processed_dir / "warehouse.duckdb"
con = duckdb.connect(str(db_path))

# 选定 schema
con.execute("CREATE SCHEMA IF NOT EXISTS analytics; SET schema='analytics';")

# ✅ 用绝对路径读 CSV，避免相对路径跑偏
raw_dir = BASE / "data" / "raw"
tables = ["customers","orders","order_items","products","events","campaigns","campaign_members"]
for name in tables:
    csv_path = (raw_dir / f"{name}.csv").as_posix()
    con.execute(f"CREATE OR REPLACE TABLE analytics.{name} AS SELECT * FROM read_csv_auto('{csv_path}', SAMPLE_SIZE=-1);")

# 读入 SQL 并生成视图/结果
rfm_sql = (BASE / "sql" / "rfm.sql").read_text()
con.execute("CREATE OR REPLACE VIEW rfm_view AS " + rfm_sql)

seg_sql = (BASE / "sql" / "segmentation.sql").read_text()
seg = con.execute(seg_sql).fetchdf()

# 看一下分群规模
seg.segment.value_counts()


segment
Regulars             1067
Loyal                 372
Champions             341
At Risk (High F)      202
Hibernating            15
Promising (Low M)       3
Name: count, dtype: int64

In [5]:
con.close()